# Query GEE for ERA5-Land wind data

In [1]:
import ee
import os
import xdem
import numpy as np
import pandas as pd
import json

In [9]:
### Define paths in directory
site_name = "Banner"
data_dir = f'/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/{site_name}'
# Reference DEM for spatial querying
if site_name=='JacksonPeak':
    refdem_fn = os.path.join(data_dir, 'refdem', 'USGS_LPC_ID_FEMAHQ_2018_D18_merged_filtered_UTM11_filled.tif')
else:
    refdem_fn = os.path.join(data_dir, 'refdem', f'{site_name}_REFDEM_WGS84.tif')
# Output file name
out_fn = os.path.join(data_dir, f'{site_name}_dominant_wind_direction_ERA5-Land_20231001-20240501.json')

In [3]:
### Initialize GEE
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
if not os.path.exists(out_fn):
    ### Create spatial bounds
    # Get DEM spatial bounds in lat lon
    refdem = xdem.DEM(refdem_fn).reproject(crs='EPSG:4326')
    bounds = refdem.bounds
    print(bounds)
    # Create polygon for spatial clipping
    region = ee.Geometry.Polygon([[bounds.left, bounds.bottom], [bounds.right, bounds.bottom],
                                [bounds.right, bounds.top], [bounds.left, bounds.top],
                                [bounds.left, bounds.bottom]])

    #### Query ERA5-Land
    bands = ['u_component_of_wind_10m', 'v_component_of_wind_10m'] # m/s
    date_range = ['2023-10-01', '2024-05-01']
    era5 = (ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
        .filterDate(date_range[0], date_range[1])
        .filterBounds(region)
        .select(bands))

    #### Calculate wind speed and direction
    def calculate_wind(img):
        u = img.select('u_component_of_wind_10m')
        v = img.select('v_component_of_wind_10m')
        speed = u.hypot(v).rename('wind_speed')
        direction = v.atan2(u).rename('wind_direction')  # Direction in radians
        return img.addBands([speed, direction])
    era5_with_wind = era5.map(calculate_wind)

    #### Filter wind speeds
    # 5 m/s = ∼minimum wind velocity for snow transport (Li and Pomeroy, 1997),
    # but no wind speeds this fast found in ERA5-Land! Use 1 m/s for filtering
    def mask_speeds(img):
        return img.updateMask(img.select('wind_speed').gte(1))
    filtered_era5 = era5_with_wind.map(mask_speeds)

    ### Calculate median wind direction
    # Upsample ERA5 data to finer resolution (~1 km) to increase the number of pixels sampled
    def upsample(img):
        return img.resample('bilinear').reproject(crs='EPSG:4326', scale=100)
    upsampled_era5 = filtered_era5.map(upsample)

    def reduce_to_median(img):
        stats = img.reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=region,
            scale=100,
            bestEffort=True
        )
        return ee.Feature(None, stats)
    reduced_collection = upsampled_era5.map(reduce_to_median)

    ### Convert to pandas DataFrame
    # Convert to a list of dictionaries for analysis
    result_list = reduced_collection.getInfo()['features']
    df = pd.DataFrame([feature['properties'] for feature in result_list])
    # Post-process: Calculate the median direction weighted by speed
    if not df.empty:
        wind_speeds = df['wind_speed'].dropna()
        wind_directions = df['wind_direction'].dropna()    
        if not wind_speeds.empty and not wind_directions.empty:
            sin_mean = np.average(np.sin(wind_directions), weights=wind_speeds)
            cos_mean = np.average(np.cos(wind_directions), weights=wind_speeds)
            dominant_direction = np.arctan2(sin_mean, cos_mean) * (180 / np.pi)  # Convert to degrees
            dominant_direction = (dominant_direction + 360) % 360  # Normalize to [0, 360]
            print(f"Dominant wind direction: {dominant_direction:.2f} degrees")
        else:
            print("No valid wind data available.")
    else:
        print("No data found for the specified area and time.")
        
    # Create dictionary of results
    results_dict = {'site_name': site_name,
                    'date_range': date_range,
                    'dominant_wind_direction_degrees': str(dominant_direction)}
    
    # Save to file
    with open(out_fn, 'w') as f:
        json.dump(results_dict, f)
    print('Dominant wind direction saved to file:', out_fn)

else:
    print('Dominant wind direction already exists in file, skipping.')